In [1]:
import numpy as np
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import string
import re
import gensim.models
import tempfile
from gensim.models import Word2Vec
import time

# Training our model

In [2]:
filter_stopwords = False
filter_pronouns = False

# Reading in our sentences from the scrape results
matthew = False
Dino = True
path = "C:/Users/laure/OneDrive/Documenten/Project 3.1 APG Files/"
filename = 'url_references_with_paragraphs_no_english.csv'
    # 'public_and_private_paragraphs_none_filtered.csv'
if matthew:
    df = pd.read_csv('DATA/scrape_results/' + filename)
elif Dino:
    df = pd.read_csv('C:/Users/01din/Desktop/data APG\FIXED_SCRAPES/' + filename)
else:
    df = pd.read_csv(path + filename)


In [3]:
# Converting to list
corpus = df['paragraphs'].tolist()

In [4]:
# Break each sentence into list of individual words
sentences = []
for sentence in corpus:
    sentences.append(sentence.split(" "))

In [5]:
# Create custom word2vec model with given parameters (from internet)
custom_model = Word2Vec(window = 10, min_count = 5, workers = 20, negative = 5, sample = 1e-5)

# Build the vocab by giving the sentences
custom_model.build_vocab(sentences, progress_per=10000)

# Train the model on the inputted sentences, epoch = 50, higher epochs give better output but increase time
start = time.time()
custom_model.train(sentences, total_examples=custom_model.corpus_count, epochs=50, report_delay=1)
end = time.time()

print("Time elapsed (seconds): " + str(end - start))

Time elapsed (seconds): 5.66746711730957


In [6]:
# Save the model for re-use
filename = 'custom_model_filtered.wordvectors'
if matthew:
    custom_model.save('DATA/Data/' + filename)
elif Dino:
    custom_model.save('C:/Users/01din/Desktop/data APG/FIXED_SCRAPES/' + filename)
else:
    custom_model.save(path + filename)

In [7]:
# Some small tests
number_of_words = str(len(custom_model.wv))
print('Number of words: ' + number_of_words)

Number of words: 6172


In [8]:
# Test to see what words are similar to pensioen
most_similar_to_pensioen = custom_model.wv.most_similar(positive='militair')
print("Words most similar to militair: ", most_similar_to_pensioen)

Words most similar to militair:  [('aanspraak', 0.8467262387275696), ('arbeidsongeschiktheid', 0.8378706574440002), ('defensie', 0.8281406760215759), ('mate', 0.8211729526519775), ('vult', 0.8176197409629822), ('ontslagen', 0.7457218766212463), ('zolang', 0.741395115852356), ('gewezen', 0.7273013591766357), ('ugm', 0.6875185370445251), ('militairen', 0.6580723524093628)]


In [9]:
tfidf_keywords = pd.read_csv('C:/Users/01din/Desktop/data APG/tfidf/tf_idf_keywords_new.csv')
keywords_tf_idf = tfidf_keywords["0"].tolist()
vocabulary1 = list(set(custom_model.wv.index_to_key))

In [10]:
difference = [x for x in keywords_tf_idf if x not in vocabulary1]
print(difference)

[]
